In [ ]:
import sys, os

from torch.utils.tensorboard import SummaryWriter
from absl import flags
from absl import app

from feature_extractor.utils import save_to_logs, get_train_dir
from feature_extractor.emb_model_lib import EmbeddingModel

In [ ]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)

In [ ]:
param = {
    "L2D": { # Parameter for Learning to defer
        "maxLabels": 16,
    },
    "TARGET": "Airspace_Opacity",
    "PATH": "../Datasets/NIH/",
    "K": 10, #Number of folds
    "LABELER_IDS": [4323195249, 4295232296],
    
    "batch_size": 64,
    "alpha": 1.0, #scaling parameter for the loss function, default=1.0
    "epochs": 150,
    "patience": 50, #number of patience steps for early stopping the training
    "expert_type": "MLPMixer", #specify the expert type. For the type of experts available, see-> models -> experts. defualt=predict
    "n_classes": 2, #K for K class classification
    "k": 0, #
    "n_experts": 2, #
    "lr": 0.001, #learning rate
    "weight_decay": 5e-4, #
    "warmup_epochs": 5, #
    "loss_type": "softmax", #surrogate loss type for learning to defer
    "ckp_dir": "./Models", #directory name to save the checkpoints
    "experiment_name": "multiple_experts", #specify the experiment name. Checkpoints will be saved with this name
    #
    "TRAIN_BATCH_SIZE": 64,
    "TEST_BATCH_SIZE": 64,
    "NUM_EXPERTS": 2,

    "GT": True, # Determines if the classifier gets all data with GT Label or only the labeld data
}

In [11]:
import neptune

import json

with open('neptune_config.json', 'r') as f:
    config = json.load(f)

config_neptune = config["neptune"]
"""run = neptune.init_run(
    project=config_neptune["project"],
    api_token=config_neptune["api_token"],
    custom_run_id="AI"
)"""

basic_Dataset = ds.BasicDataset(param["PATH"], param["TARGET"])
nih_dataloader = ds.NIH_K_Fold_Dataloader(
            dataset = basic_Dataset,
            k = param["K"],
            labelerIds = param["LABELER_IDS"],
            train_batch_size = param["TRAIN_BATCH_SIZE"],
            test_batch_size = param["TEST_BATCH_SIZE"],
            param = param
        )

train_dataloader, val_dataloader, test_dataloader = nih_dataloader.get_data_loader_for_fold(1)
dataloaders = (train_dataloader, val_dataloader, test_dataloader)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

'run = neptune.init_run(\n    project=config_neptune["project"],\n    api_token=config_neptune["api_token"],\n    custom_run_id="AI"\n)'

In [ ]:
args = {
    'dataset': "nih",
    'model': "resnet18",
    'num_classes': 2,
    'batch': 128,
    'lr': 0.001,
}

path = "../../../Datasets/NIH/"

wkdir = os.getcwd()
sys.path.append(wkdir)
    
# get training directory
train_dir = get_train_dir(wkdir, args, 'emb_net')

print("Train dir: " + train_dir)
# initialize summary writer for tensorboard
writer = SummaryWriter(train_dir + 'logs/')

# initialize base model
emb_model = EmbeddingModel(args, wkdir, writer, dataloaders, param)
# try to load previous training runs
start_epoch = emb_model.load_from_checkpoint(mode='latest')
# train model
for epoch in range(start_epoch, 10):
    # train one epoch
    loss = emb_model.train_one_epoch(epoch)
    # get validation accuracy
    valid_acc = emb_model.get_test_accuracy(return_acc=True)
    print(f'loss: {loss}')
    # save logs to json
    save_to_logs(train_dir, valid_acc, loss.item())
    # save model to checkpoint
    emb_model.save_to_checkpoint(epoch, loss, valid_acc)
# get test accuracy
emb_model.get_test_accuracy()